In [1]:
# !pip install z3

In [1]:
!pip install z3-solver


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

# Load environment variables
load_dotenv()
# Set up OpenAI API key
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["ANTHROPIC_API_KEY"] = os.getenv("ANTHROPIC_API_KEY")

https://langchain-ai.github.io/langgraph/concepts/low_level/

In [ ]:
from typing import TypedDict, List, Dict, Optional, Union
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage, SystemMessage
from langgraph.graph import StateGraph, END, START
from operator import add
from typing import Annotated
from langchain_openai import ChatOpenAI
import os

# LLM 초기화
llm = ChatOpenAI(
    temperature=0,
    model="gpt-4o"
)

# 1. 상태 정의
class LogicalAnalysisState(TypedDict):
    # 입력 텍스트
    input_text: str
    # 파악된 논리적 구문들
    logical_statements: List[str]
    # 각 구문의 진위 여부
    statement_validities: List[Dict[str, bool]]
    # 전체 신빙성 점수 (0-100)
    credibility_score: float
    # 분석 메시지들
    messages: Annotated[List[BaseMessage], add]

def analyze_context(state: LogicalAnalysisState) -> Dict:
    # 1단계: 문제 분석
    analysis_messages = [
        SystemMessage(content="""
            주어진 논리 문제를 분석하여 다음 정보를 추출하세요:
            
            1. 필요한 속성 목록:
               예시:
               - Logical (논리적)
               - Intuitive (직관적)
               - Math (수학적)
               등 문제에서 사용되는 모든 속성을 나열
            
            2. 논리적 관계 유형:
               - 단순 사실 ("X는 Y이다")
               - 조건부 관계 ("A이면 B이다")
               - 양방향 함의 ("A만이 B할 수 있다")
               - 복합 조건 ("A이고 B이면 C이다")
            
            3. 결론 명시
            
            분석 결과를 다음 형식으로 반환하세요:
            ---
            필요한 속성들:
            - 속성1 (설명)
            - 속성2 (설명)
            ...
            
            논리적 관계들:
            1. [단순 사실] X는 Y이다
            2. [조건부] A이면 B이다
            ...
            
            결론:
            Z는 W이다
            ---
        """),
        HumanMessage(content=f"다음 논리 문제를 분석해주세요:\n\n{state['input_text']}")
    ]
    
    # 분석 결과 얻기
    analysis_result = llm.invoke(analysis_messages)
    
    # 2단계: z3-solver 코드 생성
    code_messages = [
        SystemMessage(content="""
            분석된 논리 문제를 바탕으로 z3-solver 코드를 생성하세요.
            
            1. 변수 선언:
               - 분석된 모든 속성을 Bool 변수로 선언
               - 개별 인물의 변수는 생성하지 않음
            
            2. 관계 표현:
               - 단순 사실: solver.add(Y)
               - 조건부: solver.add(Implies(A, B))
               - 양방향 함의: solver.add(A == B)
               - 복합 조건: solver.add(Implies(And(A, B), C))
            
            3. 기본 논리 법칙 포함
            4. 결론의 부정 추가
            5. 검증 및 결과 출력
            
            다음 형식으로 코드를 생성하세요:
            ```python
            from z3 import *
            
            # Solver 초기화
            solver = Solver()
            
            # 변수 선언 - 모든 속성 포함
            A = Bool('A')  # 설명
            B = Bool('B')  # 설명
            ...
            
            # 기본 논리 법칙
            solver.add(Not(And(A, Not(A))))  # 모순율
            solver.add(Or(A, Not(A)))        # 배중율
            
            # 전제 추가
            solver.add(...)  # 각 전제
            
            # 결론의 부정
            solver.add(Not(...))
            
            # 검증 및 결과 출력
            result = solver.check()
            print(f"검증 결과: {result}")
            
            if result == unsat:
                print("논리적으로 타당합니다.")
                credibility = 100.0
            else:
                m = solver.model()
                print(f"논리적으로 타당하지 않습니다. 반례: {m}")
                credibility = 0.0
            
            print(f"신빙성 점수: {credibility}")
            ```
        """),
        HumanMessage(content=f"분석 결과를 바탕으로 z3-solver 코드를 생성해주세요:\n\n{analysis_result.content}")
    ]
    
    # LLM 호출
    response = llm.invoke(code_messages)
    
    # 실행 가능한 코드 블록만 추출
    code_blocks = [block.strip() for block in response.content.split('```python')]
    if len(code_blocks) > 1:
        logical_statements = [code_blocks[1].strip('`').strip()]
    else:
        logical_statements = [response.content.strip()]
    
    return {
        "logical_statements": logical_statements,
        "messages": [
            HumanMessage(content=state["input_text"]),
            AIMessage(content=f"생성된 z3-solver 코드:\n{chr(10).join(logical_statements)}")
        ]
    }
    
def validate_statements(state: LogicalAnalysisState) -> Dict:
    """각 논리적 구문의 진위 여부 평가"""
    validities = []
    total_score = 0.0
    
    try:
        # 전체 코드를 하나의 블록으로 실행
        local_vars = {}
        exec("from z3 import *", local_vars)
        
        # 모든 구문을 하나로 합침
        full_code = "\n".join(state["logical_statements"])
        exec(full_code, local_vars)
        
        # solver 객체 확인
        solver = local_vars.get('solver')
        if solver is None:
            raise ValueError("No solver object found")
            
        # 결과 확인
        check_result = solver.check()
        is_valid = check_result == local_vars['unsat']
        
        if is_valid:
            reason = "논리식이 타당합니다 (결론의 부정이 모순을 일으킴)"
            total_score = 100.0
        else:
            reason = "논리식이 타당하지 않습니다"
            total_score = 0.0
            
        validities.append({
            "statement": full_code,
            "is_valid": is_valid,
            "reason": reason
        })
        
    except Exception as e:
        validities.append({
            "statement": full_code if 'full_code' in locals() else str(state["logical_statements"]),
            "is_valid": False,
            "reason": f"코드 실행 중 오류 발생: {str(e)}"
        })
        total_score = 0.0
    
    return {
        "statement_validities": validities,
        "credibility_score": total_score,
        "messages": [AIMessage(content=f"논리 구문 평가 완료: 신빙성 점수 {total_score:.1f}/100")]
    }

def calculate_credibility(state: LogicalAnalysisState) -> Dict:
    """논리적 타당성을 기반으로 신빙성 점수 계산"""
    validities = state.get("statement_validities", [])
    
    # 각 구문의 유효성 확인
    valid_count = sum(1 for v in validities if v.get("is_valid", False))
    total_count = len(validities)
    
    # 신빙성 점수 계산 (0-100)
    if total_count > 0:
        # unsat일 때 논리적으로 타당하므로, solver.check() == unsat인 경우를 확인
        for validity in validities:
            if "solver.check() == unsat" in validity["statement"]:
                weighted_score = 100.0 if validity["is_valid"] else 0.0
                break
        else:
            weighted_score = (valid_count / total_count) * 100.0
    else:
        weighted_score = 0.0
    
    return {
        "credibility_score": weighted_score,
        "messages": [AIMessage(content=f"전체 신빙성 점수: {weighted_score:.1f}/100")]
    }

# 3. 그래프 구성
def create_analysis_graph():
    """논리 분석 그래프 생성"""
    graph = StateGraph(LogicalAnalysisState)
    
    # 노드 추가
    graph.add_node("context_analyzer", analyze_context)
    graph.add_node("statement_validator", validate_statements)
    graph.add_node("credibility_calculator", calculate_credibility)
    
    # 엣지 추가
    graph.add_edge(START, "context_analyzer")
    graph.add_edge("context_analyzer", "statement_validator")
    graph.add_edge("statement_validator", "credibility_calculator")
    graph.add_edge("credibility_calculator", END)
    
    return graph.compile()

def analyze_text_logic(text: str) -> Dict:
    """텍스트의 논리성 분석 실행"""
    graph = create_analysis_graph()
    
    # 초기 상태
    initial_state = {
        "input_text": text,
        "logical_statements": [],
        "statement_validities": [],
        "credibility_score": 0.0,
        "messages": []
    }
    
    # 그래프 실행
    result = graph.invoke(initial_state)
    
    return {
        "credibility_score": result["credibility_score"],
        "logical_statements": result["logical_statements"],
        "validities": result["statement_validities"],
        "analysis_log": [m.content for m in result["messages"]]
    }

# 실행 예시
if __name__ == "__main__":
    test_text = """
# 핵심 규칙
	##라운드
	- 게임은 최대 5라운드로 구성됩니다.
	- 각 라운드에서는 문제가 출력되고 사용자가 답변을 제출합니다.
	- 하나의 라운드는 '새로운 문제'가 출력되어 '다음 번 새로운 문제'가 출력될 때 사이의 기간입니다.
	- 정답을 맞추거나 **한 라운드 안에서 오답을 두 번 제출하면 다음 라운드**로 넘어갑니다.
		- 한 라운드 안에서 오답을 한 번 제출하면 라운드가 넘어가지 않으며, 힌트가 제공됩니다.
	##정오판별
	- 정답은 대소문자를 구분하지 않습니다.
	- 한글 입력은 무조건 오답으로 처리됩니다.
	- 입력된 영어 단어를 정답과 비교합니다.
	- 입력된 모든 메시지는 정답 판정을 실행합니다.
	##정답기록
	- 정답을 맞출 때마다 answer_count가 1씩 증가합니다.
	##출력
	- 모든 게임 데이터는 JSON 형식으로 출력됩니다.
	##종료
	- 게임은 5라운드를 모두 진행하거나, 3개의 정답을 맞추면 종료됩니다.
		- 5라운드에 맞춘 답의 수가 3보다 작으면 실패 엔딩입니다.
		- 맞춘 정답이 3개 이상이면 성공 엔딩입니다.
	- 게임이 종료되는 조건은 total_round가 5 이상이거나 answer_count가 3 이상일 때입니다.
	- 게임 종료 시 마지막 라운드의 정답 판정 규칙이 적용되며, 정답 카운트는 이전 상태값을 유지합니다.
	- 5라운드에 도달했을 때, 정답 수가 0인 경우에도 정상적으로 처리하며, 마지막 입력이 오답이면 check_answer는 false로 설정됩니다.
	- 게임 종료 메시지는 실제 정답 수를 기준으로 출력됩니다.
    """
    
    result = analyze_text_logic(test_text)
    
    print(f"신빙성 점수: {result['credibility_score']:.1f}/100")
    print("\n논리 구문들:")
    for stmt in result["logical_statements"]:
        print(f"- {stmt}")
    # print("\n분석 로그:")
    # for log in result["analysis_log"]:
    #     print(f"- {log}")

검증 결과: unsat
논리적으로 타당합니다.
신빙성 점수: 100.0
신빙성 점수: 100.0/100

논리 구문들:
- from z3 import *

# Solver 초기화
solver = Solver()

# 변수 선언 - 모든 속성 포함
Philosopher = Bool('Philosopher')  # 철학자
Speculative = Bool('Speculative')  # 사변적
Analytical = Bool('Analytical')    # 분석적
Intuitive = Bool('Intuitive')      # 직관적
Insightful = Bool('Insightful')    # 통찰적
Abstract = Bool('Abstract')        # 추상적
Conceptual = Bool('Conceptual')    # 개념적
Systematic = Bool('Systematic')    # 체계적
Metacognitive = Bool('Metacognitive')  # 메타인지적
Critical = Bool('Critical')        # 비판적
Reflective = Bool('Reflective')    # 반성적
Rational = Bool('Rational')        # 합리적
Transcendent = Bool('Transcendent')  # 초월적

# 기본 논리 법칙
solver.add(Not(And(Philosopher, Not(Philosopher))))  # 모순율
solver.add(Or(Philosopher, Not(Philosopher)))        # 배중율

# 전제 추가
solver.add(Implies(Philosopher, Or(Speculative, Analytical)))  # 모든 철학자는 사변적이거나 분석적이다
solver.add(Implies(And(Speculative, Intuitive), Insightful))   # 사변적인 사람이 직관적이면 통찰적이다
solver.add